In [ ]:
pip install jovian --quiet

     |████████████████████████████████| 68 kB 813 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install jovian

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#library imports
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy import sparse
import sys
import jovian

In [ ]:
!unzip '/content/drive/MyDrive/GdriveXbot/Indus_OS_Participants_Data.zip'
%cd "Indus_OS_Participants_Data"

Archive:  /content/drive/MyDrive/GdriveXbot/Indus_OS_Participants_Data.zip
   creating: Indus_OS_Participants_Data/
  inflating: __MACOSX/._Indus_OS_Participants_Data  
  inflating: Indus_OS_Participants_Data/IndusOS_Data Dictionary.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._IndusOS_Data Dictionary.csv  
  inflating: Indus_OS_Participants_Data/app_installs.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._app_installs.csv  
  inflating: Indus_OS_Participants_Data/.DS_Store  
  inflating: __MACOSX/Indus_OS_Participants_Data/._.DS_Store  
  inflating: Indus_OS_Participants_Data/validation_data.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._validation_data.csv  
  inflating: Indus_OS_Participants_Data/actual_set.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._actual_set.csv  
  inflating: Indus_OS_Participants_Data/app_metadata.csv  
  inflating: __MACOSX/Indus_OS_Participants_Data/._app_metadata.csv  
  inflating: Indus_OS_Participants_Data/app_usa

In [ ]:
%cd "Indus_OS_Participants_Data"

/content/Indus_OS_Participants_Data


In [ ]:
actual_set = pd.read_csv('actual_set.csv')
app_installs = pd.read_csv('app_installs.csv')
app_metadata = pd.read_csv('app_metadata.csv')
app_usage = pd.read_csv('app_usage.csv')
user_metadata = pd.read_csv('user_metadata.csv')
validation_data = pd.read_csv('validation_data.csv')

In [ ]:
user_metadata.head()

,uid,device,device_category,state,city,network_type,user_lang,space_available
0,94698,SM-M215F,Mid,Assam,Dibrugarh,4G,en-US,94.32
1,257076,SM-J400F,Mass,West Bengal,Siliguri,NaN,NaN,NaN
2,283805,SM-J415F,Mass,Jharkhand,Deogarh,wifi,en-US,11.45
3,248262,SM-A336E,High,Gujarat,Ahmedabad,4G,en-US,91.20
4,283806,SM-M205F,Mid,National Capital Territory of Delhi,Delhi,4G,en-US,42.62


In [ ]:
actual_set.head()

,uid,item_id
0,177489,601235
1,14644,64855
2,14699,64855
3,190856,64855
4,70394,64866


In [ ]:
validation_data.head()

,uid
0,255352
1,290183
2,210428
3,17401
4,48078


In [ ]:
validation_data.shape

(42252, 1)

In [ ]:
app_usage = app_usage.drop('app_use_date', axis =1)

In [ ]:
app_usage.head()

,uid,item_id,time_spent
0,185459,601235,2180211
1,164721,601235,4850939
2,22949,601235,1026713
3,1773,601235,419773
4,12539,601235,1609444


In [ ]:
#Filtering users who spent time atleast more than 10 seconds
app_usage = app_usage.loc[app_usage['time_spent'] > 10000] 

#Converting milliseconds to minutes
app_usage['time_spent'] = app_usage['time_spent'].div(60000).round(2)

app_usage.head(5)

,uid,item_id,time_spent
0,185459,601235,36.34
1,164721,601235,80.85
2,22949,601235,17.11
3,1773,601235,7.00
4,12539,601235,26.82


In [ ]:
app_usage.rename(columns = {'time_spent':'rating'}, inplace = True) 

In [ ]:
app_usage.head(5)

,uid,item_id,rating
0,185459,601235,36.34
1,164721,601235,80.85
2,22949,601235,17.11
3,1773,601235,7.00
4,12539,601235,26.82


In [ ]:
app_usage.isnull().sum()

uid        0
item_id    0
rating     0
dtype: int64

In [ ]:
Counter(app_usage.rating)

Counter({36.34: 285,
         80.85: 109,
         17.11: 749,
         7.0: 1863,
         26.82: 441,
         14.88: 874,
         25.61: 467,
         6.85: 1872,
         36.51: 277,
         0.89: 9987,
         1.74: 6374,
         1.23: 8285,
         0.96: 9630,
         0.34: 16738,
         3.33: 3790,
         6.42: 2108,
         0.18: 24403,
         7.19: 1753,
         0.58: 13389,
         0.65: 12525,
         1.0: 9434,
         0.54: 14657,
         0.99: 9502,
         1.35: 7702,
         12.02: 1096,
         0.51: 14319,
         7.14: 1915,
         10.98: 1230,
         2.72: 4449,
         17.62: 787,
         5.54: 2320,
         2.34: 5227,
         12.89: 1056,
         0.42: 14971,
         0.93: 9891,
         2.35: 5127,
         52.42: 198,
         1.03: 9461,
         14.04: 944,
         0.2: 22879,
         3.9: 3214,
         8.49: 1514,
         1.63: 6750,
         5.72: 2272,
         1.28: 8037,
         2.16: 5450,
         5.51: 2277,
      

In [ ]:
Counter(app_usage.groupby(['uid']).count()['item_id'])

Counter({26: 5518,
         6: 3730,
         20: 5782,
         28: 5441,
         83: 179,
         23: 5753,
         3: 2630,
         41: 3249,
         86: 137,
         71: 426,
         16: 5468,
         40: 3416,
         27: 5367,
         34: 4369,
         17: 5539,
         30: 4982,
         66: 622,
         76: 299,
         2: 2391,
         18: 5719,
         42: 3058,
         54: 1533,
         95: 89,
         51: 1795,
         31: 4982,
         49: 2074,
         22: 5719,
         7: 4134,
         24: 5644,
         37: 3984,
         13: 5045,
         39: 3604,
         12: 4930,
         9: 4332,
         19: 5665,
         35: 4243,
         14: 5330,
         38: 3816,
         62: 827,
         4: 2960,
         11: 4653,
         36: 4215,
         50: 2008,
         48: 2310,
         25: 5591,
         45: 2666,
         21: 5719,
         8: 4119,
         15: 5555,
         57: 1270,
         98: 61,
         33: 4622,
         29: 5198,
         4

In [ ]:
#Average number of ratings per user
np.mean(app_usage.groupby(['uid']).count()['item_id'])

28.649443781612867

In [ ]:
train_df, valid_df = train_test_split(app_usage, test_size=0.2)

#resetting indices to avoid indexing errors in the future
train_df = train_df.reset_index()[['uid', 'item_id', 'rating']]
valid_df = valid_df.reset_index()[['uid', 'item_id', 'rating']]

In [ ]:
train_df.head()

,uid,item_id,rating
0,300577,69902,46.98
1,298836,20875,62.50
2,159823,20875,48.30
3,132839,732171,37.23
4,159581,797459,5.68


In [ ]:
app_usage.head()

,uid,item_id,rating
0,185459,601235,36.34
1,164721,601235,80.85
2,22949,601235,17.11
3,1773,601235,7.00
4,12539,601235,26.82


In [ ]:
actual_set['rating'] = 100

In [ ]:
ft = pd.concat([app_usage,actual_set],axis=0)
ft.head()

,uid,item_id,rating
0,185459,601235,36.34
1,164721,601235,80.85
2,22949,601235,17.11
3,1773,601235,7.00
4,12539,601235,26.82


In [ ]:
ft.tail()

,uid,item_id,rating
326241,296038,141827,100.0
326242,168106,153823,100.0
326243,112134,776536,100.0
326244,130065,862905,100.0
326245,219036,683417,100.0


In [ ]:
ft.shape

(7145845, 3)

In [ ]:
print('Number of unique books: ', app_usage['item_id'].nunique())
print('Number of unique users: ', app_usage['uid'].nunique())

Number of unique books:  47052
Number of unique users:  238036


In [ ]:
print('Number of unique books: ', ft['item_id'].nunique())
print('Number of unique users: ', ft['uid'].nunique())

In [ ]:
def encode_column(column):
    """ Encodes a pandas column with continous IDs"""
    keys = column.unique()
    key_to_id = {key:idx for idx,key in enumerate(keys)}
    return key_to_id, np.array([key_to_id[x] for x in column]), len(keys)

In [ ]:
def encode_df(user_df):
    """Encodes rating data with continuous user and an item ids"""
    
    item_ids, user_df['item_id'], num_item = encode_column(user_df['item_id'])
    user_ids, user_df['uid'], num_users = encode_column(user_df['uid'])
    return user_df, num_users, num_item, user_ids, item_ids

In [ ]:
user_df, num_users, num_item, user_ids, item_ids = encode_df(app_usage)
print("Number of users :", num_users)
print("Number of apps :", num_item)

Number of users : 238036
Number of apps : 47052


In [ ]:
print("Number of users :",user_df['uid'].nunique())
print("Number of apps :",user_df['item_id'].nunique())

Number of users : 238036
Number of apps : 47052


In [ ]:
app_usage.head()

,uid,item_id,rating
0,0,0,36.34
1,1,0,80.85
2,2,0,17.11
3,3,0,7.00
4,4,0,26.82


In [ ]:
user_df.head()

,uid,item_id,rating
0,0,0,36.34
1,1,0,80.85
2,2,0,17.11
3,3,0,7.00
4,4,0,26.82


In [ ]:
print('Number of unique books: ', app_usage['item_id'].nunique())
print('Number of unique users: ', app_usage['uid'].nunique())

Number of unique books:  47052
Number of unique users:  238036


In [ ]:
app_usage.sort_values(by=['item_id','uid'])

In [ ]:
train_df.head()

In [ ]:
train_df.head()

In [ ]:
user_df, num_users, num_item, user_ids, item_ids = encode_df(train_df)
print("Number of users :", num_users)
print("Number of apps :", num_item)
user_df.head()

Number of users : 237450
Number of apps : 43535


,uid,item_id,rating
0,0,0,8.50
1,1,1,8.90
2,2,2,111.31
3,3,3,42.79
4,4,4,0.67


In [ ]:
def create_embeddings(n, K):
    """
    Creates a random numpy matrix of shape n, K with uniform values in (0, 11/K)
    n: number of items/users
    K: number of factors in the embedding 
    """
    return 11*np.random.random((n, K)) / K

In [ ]:
def create_sparse_matrix(df, rows, cols, column_name="rating"):
    """ Returns a sparse utility matrix""" 
    return sparse.csc_matrix((df[column_name].values,(df['uid'].values, df['item_id'].values)),shape=(rows, cols))

In [ ]:
user_df, num_users, num_item, user_ids, item_ids = encode_df(train_df)
Y = create_sparse_matrix(user_df, num_users, num_item)

In [ ]:
Y.todense()

matrix([[21.81,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  6.23,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  3.68, ...,  0.  ,  0.  ,  0.  ],
        ...,
        [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [ ]:
def predict(df, emb_user, emb_item):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Computes df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    df['prediction'] = np.sum(np.multiply(emb_item[df['item_id']],emb_user[df['uid']]), axis=1)
    return df

In [ ]:
lmbda = 0.0002

In [ ]:
def cost(df, emb_user, emb_item):
    """ Computes mean square error"""
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_item.shape[0])
    predicted = create_sparse_matrix(predict(df, emb_user, emb_item), emb_user.shape[0], emb_item.shape[0], 'prediction')
    return np.sum((Y-predicted).power(2))/df.shape[0] 

In [ ]:
def gradient(df, emb_user, emb_item):
    """ Computes the gradient for user and item embeddings"""
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_item.shape[0])
    predicted = create_sparse_matrix(predict(df, emb_user, emb_item), emb_user.shape[0], emb_item.shape[0], 'prediction')
    delta =(Y-predicted)
    grad_user = (-2/df.shape[0])*(delta*emb_item) + 2*lmbda*emb_user
    grad_item = (-2/df.shape[0])*(delta.T*emb_user) + 2*lmbda*emb_item
    return grad_user, grad_item

In [ ]:
def gradient_descent(df, emb_user, emb_item, iterations=2000, learning_rate=0.01, df_val=None):
    """ 
    Computes gradient descent with momentum (0.9) for given number of iterations.
    emb_user: the trained user embedding
    emb_item: the trained item embedding
    """
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_item.shape[0])
    beta = 0.9
    grad_user, grad_item = gradient(df, emb_user, emb_item)
    v_user = grad_user
    v_item = grad_item
    for i in range(iterations):
        grad_user, grad_item = gradient(df, emb_user, emb_item)
        v_user = beta*v_user + (1-beta)*grad_user
        v_item = beta*v_item + (1-beta)*grad_item
        emb_user = emb_user - learning_rate*v_user
        emb_item = emb_item - learning_rate*v_item
        if(not (i+1)%50):
            print("\niteration", i+1, ":")
            print("train mse:",  cost(df, emb_user, emb_item))
            if df_val is not None:
                print("validation mse:",  cost(df_val, emb_user, emb_item))
    return emb_user, emb_item

In [ ]:
emb_user = create_embeddings(num_users, 32)
emb_item = create_embeddings(num_item, 32)
emb_user, emb_item = gradient_descent(user_df, emb_user, emb_item, iterations=800, learning_rate=1)


iteration 50 :
train mse: 6328.009947318592

iteration 100 :
train mse: 5537.419374372136

iteration 150 :
train mse: 5212.5623340748125

iteration 200 :
train mse: 4999.519134150636

iteration 250 :
train mse: 4658.7366657261655

iteration 300 :
train mse: 4040.025372168941

iteration 350 :
train mse: 3694.0670802634086

iteration 400 :
train mse: 3426.3797921165465

iteration 450 :
train mse: 3212.899372675388

iteration 500 :
train mse: 3064.1949312733327

iteration 550 :
train mse: 2932.014434151851

iteration 600 :
train mse: 2811.0204215778017

iteration 650 :
train mse: 2701.941359404625

iteration 700 :
train mse: 2604.6250187161704

iteration 750 :
train mse: 2517.136383796875

iteration 800 :
train mse: 2435.6346800297147


In [ ]:
def call_item_item(self, product):
        product_x = self.product_table.lookup(product)
        pe = tf.expand_dims(self.product_embedding(product_x), 0)
        
        all_pe = tf.expand_dims(self.product_embedding.embeddings, 0)#note this only works if the layer has been built!
        scores = tf.reshape(self.dot([pe, all_pe]), [-1])
        
        top_scores, top_indices = tf.math.top_k(scores, k=100)
        top_ids = tf.gather(self.products, top_indices)
        return top_ids, top_scores

In [ ]:
def predict(df, emb_user, emb_item):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Computes df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    df['prediction'] = np.sum(np.multiply(emb_item[df['item_id']],emb_user[df['uid']]), axis=1)
    return df

array([0.24845623, 0.24100342, 0.15620079, 0.20366953, 0.10066095,
       0.10641945, 0.0673662 , 0.17111865, 0.06793226, 0.16289183,
       0.15887899, 0.0388557 , 0.10345477, 0.12284483, 0.05811157,
       0.04604085, 0.16566006, 0.24844094, 0.01242908, 0.15690099,
       0.10311554, 0.06308456, 0.22703793, 0.2520366 , 0.1617025 ,
       0.05335531, 0.09081098, 0.17554054, 0.09443465, 0.05389565,
       0.11306009, 0.0525177 ])

In [ ]:
emb_user = create_embeddings(num_users, 32)
emb_item = create_embeddings(num_item, 32)
emb_user, emb_item = gradient_descent(user_df, emb_user, emb_item, iterations=800, learning_rate=1)


iteration 50 :
train mse: 5343.329580210165

iteration 100 :
train mse: 4661.629859370922

iteration 150 :
train mse: 4381.703557377513

iteration 200 :
train mse: 4194.6793003111425

iteration 250 :
train mse: 3886.717632628585

iteration 300 :
train mse: 3384.416400990598

iteration 350 :
train mse: 3133.818247174303

iteration 400 :
train mse: 2918.6511701590243

iteration 450 :
train mse: 2725.921944959832

iteration 500 :
train mse: 2592.798275600406

iteration 550 :
train mse: 2475.2105071869814

iteration 600 :
train mse: 2363.143835977539

iteration 650 :
train mse: 2258.7498407639446

iteration 700 :
train mse: 2165.071735562239

iteration 750 :
train mse: 2082.2538519964937

iteration 800 :
train mse: 2006.8346482695902


In [ ]:
def encode_new_data(valid_df, user_ids, item_ids):
    """ Encodes valid_df with the same encoding as train_df.
    """
    df_val_chosen = valid_df['item_id'].isin(item_ids.keys()) & valid_df['uid'].isin(user_ids.keys())
    valid_df = valid_df[df_val_chosen]
    valid_df['item_id'] =  np.array([item_ids[x] for x in valid_df['item_id']])
    valid_df['uid'] = np.array([user_ids[x] for x in valid_df['uid']])
    return valid_df

In [ ]:
print("before encoding:", valid_df.shape)
valid_df = encode_new_data(valid_df, user_ids, item_ids)
print("after encoding:", valid_df.shape)

before encoding: (1363920, 3)
after encoding: (1363920, 3)


In [ ]:
train_mse = cost(user_df, emb_user, emb_item)
val_mse = cost(valid_df, emb_user, emb_item)
print(train_mse, val_mse)

2435.6346800297147 1467.088223123895


In [ ]:
valid_df.head()

,uid,item_id,rating,prediction
0,28014,7300,3.58,1.793976
1,77390,31301,0.26,4.268633
2,65298,37931,1.26,3.846285
3,157265,7144,6.48,5.634718
4,73261,42719,1.35,11.334733


In [ ]:
actual_set.head()

,uid,item_id
0,177489,601235
1,14644,64855
2,14699,64855
3,190856,64855
4,70394,64866


In [ ]:
actual_set_e = encode_new_data(actual_set, user_ids, item_ids)
#actual_set_e.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
actual_set_e.head()

,uid,item_id
0,20239,0
6,90552,2
7,137763,2
8,88032,2
9,46,2


In [ ]:
predict(actual_set_e, emb_user, emb_item)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,uid,item_id,prediction
0,20239,0,0.439682
6,90552,2,0.215258
7,137763,2,0.388469
8,88032,2,0.426794
9,46,2,0.430698
...,...,...,...
326237,93881,46812,1.485674
326238,185060,47047,1.503117
326239,93075,47048,1.341935
326240,193840,47048,1.451466


In [ ]:
actual_set_e

In [ ]:
cost(actual_set_e, emb_user, emb_item)

2143.2304276251843

In [ ]:
actual_set_e.head()

,uid,item_id,prediction
0,20239,0,0.439682
6,90552,2,0.215258
7,137763,2,0.388469
8,88032,2,0.426794
9,46,2,0.430698


In [ ]:
actual_set_e.head()

,uid,item_id,prediction
0,20239,0,0.439682
6,90552,2,0.215258
7,137763,2,0.388469
8,88032,2,0.426794
9,46,2,0.430698


In [ ]:
actual_set_e=actual_set_e.reset_index(drop=True)

In [ ]:
actual_set_e['rating'] = 50

In [ ]:
actual_set.head()

,uid,item_id
0,177489,601235
1,14644,64855
2,14699,64855
3,190856,64855
4,70394,64866


In [ ]:
actual_set_e.head()

,uid,item_id,rating
0,20239,0,50
1,90552,2,50
2,137763,2,50
3,88032,2,50
4,46,2,50


In [ ]:
app_usage.head()

,uid,item_id,rating,prediction
0,0,0,36.34,0.667121
1,1,0,80.85,0.268946
2,2,0,17.11,0.677145
3,3,0,7.00,0.274318
4,4,0,26.82,0.215590


In [ ]:
aa.head()

NameError: ignored

In [ ]:
full = pd.concat([app_usage,actual_set],axis=0)

In [ ]:
full=full.reset_index(drop=True)

In [ ]:
full.head()

,uid,item_id,rating,prediction
0,0,0,36.34,0.667121
1,1,0,80.85,0.268946
2,2,0,17.11,0.677145
3,3,0,7.00,0.274318
4,4,0,26.82,0.215590


In [ ]:
full.uid.nunique()

273538

In [ ]:
full.item_id.nunique()

70663

In [ ]:
app_u_new=full.groupby(['uid','item_id'])['rating'].agg(['sum'])
app_u_new=app_u_new.reset_index(drop=False)

In [ ]:
app_u_new.head()

,uid,item_id,sum
0,0,0,36.34
1,0,2960,62.38
2,0,6030,15.97
3,0,6361,82.26
4,0,9175,71.22


In [ ]:
app_u_new[app_u_new.uid==2]

In [ ]:
app_u_new[app_u_new.uid==2]

,uid,item_id,sum
0,2,5526,124.29
1,2,20875,6.44
2,2,25349,6.91
3,2,29763,0.36
4,2,55857,50.00
5,2,145082,4.03
6,2,145569,1.06
7,2,146521,0.22
8,2,151402,0.64
9,2,589378,8.28


In [ ]:
print('Number of unique books: ', app_u_new['item_id'].nunique())
print('Number of unique users: ', app_u_new['uid'].nunique())

Number of unique books:  70663
Number of unique users:  273538


In [ ]:
full = user_df.copy()

In [ ]:
full=full[['uid','item_id','rating']]

In [ ]:
full.head()

,uid,item_id,rating
0,0,0,36.34
1,1,0,80.85
2,2,0,17.11
3,3,0,7.00
4,4,0,26.82


In [ ]:
full.item_id.max()

47051

In [ ]:
app_count = (full.
     groupby(by = ['item_id'])['uid'].
     count().
     reset_index().
     rename(columns = {'uid': 'app_count'})
     [['item_id', 'app_count']]
    )
app_count.head()

,item_id,app_count
0,0,11
1,1,1
2,2,75
3,3,3
4,4,1


In [ ]:
rating_with_totalRatingCount = full.merge(app_count, left_on = 'item_id', right_on = 'item_id', how = 'left')
rating_with_totalRatingCount.head()

,uid,item_id,rating,app_count
0,0,0,36.34,11
1,1,0,80.85,11
2,2,0,17.11,11
3,3,0,7.00,11
4,4,0,26.82,11


In [ ]:
popularity_threshold = 30
rating_popular_movie= rating_with_totalRatingCount.query('app_count >= @popularity_threshold')
rating_popular_movie.head()

,uid,item_id,rating,app_count
12,11,2,0.96,75
13,12,2,0.34,75
14,13,2,3.33,75
15,14,2,6.42,75
16,15,2,0.18,75


In [ ]:
user_count=(full.
     groupby(by = ['uid'])['item_id'].
     count().
     reset_index().
     rename(columns = {'item_id': 'total_user_Count'})
     [['uid', 'total_user_Count']]
    )
user_count.head()

,uid,total_user_Count
0,0,37
1,1,43
2,2,52
3,3,94
4,4,72


In [ ]:
mine = rating_popular_movie.merge(user_count, left_on = 'uid', right_on = 'uid', how = 'left')

In [ ]:
user_threshold = 2
mine= mine.query('total_user_Count >= @user_threshold')
mine.head()

,uid,item_id,rating,app_count,total_user_Count
0,11,2,0.96,75,31
1,12,2,0.34,75,14
2,13,2,3.33,75,43
3,14,2,6.42,75,40
4,15,2,0.18,75,12


In [ ]:
mine =  mine[['uid','item_id','rating','app_count']]

In [ ]:
print('Number of unique books: ', mine['item_id'].nunique())
print('Number of unique users: ', mine['uid'].nunique())

Number of unique books:  5968
Number of unique users:  235663


In [ ]:
validation_data.shape

(42252, 2)

In [ ]:
validation_data.head()

,uid
0,255352
1,290183
2,210428
3,17401
4,48078


In [ ]:
validation_data['item_id'] = 601235

In [ ]:
validation_data.shape

(42252, 2)

In [ ]:
validation_data_new =  encode_new_data(validation_data, user_ids, item_ids)

In [ ]:
validation_data_new.head()

,uid,item_id
0,187884,0
1,20087,0
2,105849,0
3,92084,0
4,46430,0


In [ ]:
validation_data_new.shape

(42038, 2)

In [ ]:
validation_data_new['uid'].max()

238014

In [ ]:
validation_data_new.uid.nunique()

42038

In [ ]:
test_df=mine[mine.uid.isin(validation_data_new['uid']) == True]

In [ ]:
test_df.head()

,uid,item_id,rating,app_count
0,11,2,0.96,75
3,14,2,6.42,75
4,15,2,0.18,75
5,15,2,7.19,75
15,20,2,10.98,75


In [ ]:
test_df = test_df.reset_index(drop=True)

In [ ]:
print('Number of unique books: ', test_df['item_id'].nunique())
print('Number of unique users: ', test_df['uid'].nunique())

Number of unique books:  5900
Number of unique users:  41809


In [ ]:
print('Number of unique books: ', test_df['item_id'].nunique())
print('Number of unique users: ', test_df['uid'].nunique())

Number of unique books:  7303
Number of unique users:  41940


In [ ]:
test_df_n = test_df[test_df.app_count>90]

In [ ]:
test_df_n = test_df_n.reset_index(drop=True)
test_df_n.head()

,uid,item_id,rating,app_count
0,138,34,0.78,130
1,138,34,1.62,130
2,150,34,0.57,130
3,151,34,0.91,130
4,151,34,0.56,130


In [ ]:
test_df = test_df.reset_index(drop=True)

In [ ]:
print('Number of unique books: ', test_df_n['item_id'].nunique())
print('Number of unique users: ', test_df_n['uid'].nunique())

Number of unique books:  2988
Number of unique users:  41806


In [ ]:
print('Number of unique books: ', test_df['item_id'].nunique())
print('Number of unique users: ', test_df['uid'].nunique())

Number of unique books:  5900
Number of unique users:  41809


In [ ]:
test_df.head()

,uid,item_id,rating,app_count
0,11,2,0.96,75
1,14,2,6.42,75
2,15,2,0.18,75
3,15,2,7.19,75
4,20,2,10.98,75


In [ ]:
ty=test_df.uid.unique()

In [ ]:
len(ty)

41809

In [ ]:
validation_data_new[validation_data_new.uid==20087]

In [ ]:
ty

array([187884,  20087, 105849, ..., 163968,  65369,  11281])

In [ ]:
rt = test_df_n.item_id.unique()

In [ ]:
len(rt)

2988

In [ ]:
ss_uid = []
ss_item = []
for i in ty[0:10000]:
  dd = full[full.uid==i]
  dd = dd.item_id.values
  cc=test_df_n[test_df_n.item_id.isin(dd) == False]
  cc = cc.item_id.unique()
  for j in cc:
    ss_uid.append(i)
    ss_item.append(j)
  #break  

In [ ]:
nn_new=pd.DataFrame()
nn_new['uid']=ss_uid
nn_new['item_id']=ss_item
#nn_new['rating'] = 100

In [ ]:
nn_new.head()

,uid,item_id
0,11,34
1,11,38
2,11,101
3,11,112
4,11,117


In [ ]:
nn_new_1 = predict(nn_new, emb_user, emb_item)

In [ ]:
nn_new.shape

(29743489, 2)

In [ ]:
nn_new.shape

(44670764, 3)

In [ ]:
nn_new[nn_new.uid==20087]

,uid,item_id,rating
12767203,20087,34,100
12767204,20087,38,100
12767205,20087,101,100
12767206,20087,112,100
12767207,20087,117,100
...,...,...,...
12770668,20087,818431,100
12770669,20087,818017,100
12770670,20087,71626,100
12770671,20087,154052,100


In [ ]:
nn_new.head()

,uid,item_id,rating
0,187884,34,100
1,187884,38,100
2,187884,101,100
3,187884,112,100
4,187884,117,100


In [ ]:
nn_new = nn_new.reset_index(drop=True)

In [ ]:
#test_df_n = encode_new_data(nn_new, user_ids, item_ids)
cost(nn_new, emb_user, emb_item)

In [ ]:
nn_new.head()

In [ ]:
c_p = test_df_n.set_index(['item_id']).groupby('uid')['rating_preds'].nlargest(4).reset_index() 
c_p.head()

In [ ]:
test_df_n.head()

In [ ]:
validation_data.head()

,uid
0,255352
1,290183
2,210428
3,17401
4,48078


In [ ]:
df_n=user_df[user_df.uid.isin(validation_data['uid']) == True]

In [ ]:
test_df.describe()

,uid,item_id,rating,prediction
count,312344.000000,312344.000000,312344.0,312344.000000
mean,113844.549279,3233.257236,50.0,11.210350
std,67591.914610,6404.856041,0.0,25.662471
min,0.000000,0.000000,50.0,-67.835467
25%,55191.000000,63.000000,50.0,0.651094
50%,111902.000000,584.000000,50.0,2.120542
75%,171175.500000,3154.000000,50.0,10.766228
max,237409.000000,43504.000000,50.0,1811.887947


In [ ]:
actual_set.head()

,uid,item_id
0,177489,601235
1,14644,64855
2,14699,64855
3,190856,64855
4,70394,64866


In [ ]:
valid_df.head()

,uid,item_id,rating,prediction
0,167970,17,38.42,17.934124
1,98606,144,2.37,3.635008
2,147064,55,107.64,13.179147
3,123734,1006,8.30,0.244380
4,22509,3,4.08,5.738667


In [ ]:
sub = valid_df[['uid','item_id']].copy()
sub.head()

,uid,item_id
0,165061,25
1,124372,1
2,87422,24
3,206650,20
4,114656,13


In [ ]:
sub = sub[sub.uid.isin(validation_data.uid)].astype({"uid":'object', "item_id":'object'})

In [ ]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190791 entries, 1 to 1363917
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   uid      190791 non-null  object
 1   item_id  190791 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [ ]:
sub = sub.astype({"uid":'object', "item_id":'object'})



In [ ]:
sub.to_csv("submission_from_starter_script.csv", index = False, header = True)

In [ ]:
import pandas as pd
sample_submission=pd.read_csv('sample_submission.csv')
stest=pd.read_csv('submission_from_starter_script.csv')
common = sample_submission.merge(stest, on=["uid"])
result = sample_submission[~sample_submission.uid.isin(common.uid)]
new=pd.concat([stest, result], ignore_index = True)
new.to_csv("sim_note.csv",index=False)

In [ ]:
df_n=df[df.uid.isin(actual_set['uid']) == True]

In [ ]:
actual_set